In [1]:
import numpy as np
import pandas as pd

In [2]:
scorecard = pd.read_csv('Most-Recent-Cohorts-Scorecard-Elements.csv')

In [3]:
#scorecard.head()

scorecard.shape

(7175, 124)

In [4]:
salaries = pd.read_csv('salaries-by-region.csv')

In [5]:
#salaries.head()
salaries.shape

(320, 8)

In [6]:
scorecard.rename(columns={'INSTNM':'School Name'}, inplace=True)

In [7]:
#plain inner join without doing any formatting on the join columns, just to see how many matches before doing any cleansing
plainjoin = pd.merge(salaries,
                 scorecard,
                 on='School Name', 
                 how='inner')

plainjoin.shape

(148, 131)

In [8]:
#formatting step 1: remove all characters within parenthesis
salaries['School Name Extract 1'] = salaries['School Name'].str.replace(r"\(.*\)","")

In [9]:
#formatting step 2: replace all commas with hyphens
salaries['School Name Extract 2'] = salaries['School Name Extract 1'].str.replace(r",","-")

In [10]:
#formatting step 3: replace all (zero or more spaces - hyphen - zero or more spaces) with hyphen (without spaces)
salaries['Formatted School Name'] = salaries['School Name Extract 2'].str.replace(r"[\s]*-[\s]*","-")

In [11]:
#rename the join column on the scorecard dataset also to do another join
scorecard.rename(columns={'School Name':'Formatted School Name'}, inplace=True)

In [12]:
#format the 'Mid-Career Median Salary' column
salaries['Mid-Career Median Salary'] = salaries['Mid-Career Median Salary'].str.replace("$","")
salaries['Mid-Career Median Salary'] = salaries['Mid-Career Median Salary'].str.replace(",","")
salaries['Mid-Career Median Salary'] = salaries['Mid-Career Median Salary'].str.replace(".00","")

In [13]:
salaries['Salary'] = pd.to_numeric(salaries['Mid-Career Median Salary'])

In [14]:
salaries['Salary']

0      120
1      120
2      120
3      110
4      100
5      100
6      100
7      100
8      100
9       99
10      99
11      96
12      95
13      90
14      88
15      80
16      86
17      85
18      84
19      84
20      84
21      82
22      81
23      80
24      72
25      71
26      71
27      67
28     100
29      97
      ... 
290     82
291     82
292     81
293     81
294     80
295     80
296      0
297     79
298     78
299     78
300     78
301     78
302     77
303     77
304     76
305     76
306     76
307     75
308     74
309     74
310     70
311     70
312     72
313     72
314     72
315     70
316     69
317     66
318     63
319     62
Name: Salary, dtype: int64

In [42]:
#Create new column SalaryClass to make the Salary ranges into classes
salaries['SalaryClass'] = 100
salaries.loc[salaries['Salary'].between(0, 75, inclusive=False), 'SalaryClass'] = 0
salaries.loc[salaries['Salary'].between(75, 90, inclusive=True), 'SalaryClass'] = 1
salaries.loc[salaries['Salary'].between(90, 500, inclusive=False), 'SalaryClass'] = 2

In [44]:
rowstodrop = salaries[ salaries['SalaryClass'] == 100 ].index
 
# Delete these row indexes from dataFrame
salaries.drop(rowstodrop , inplace=True)

In [45]:
salaries['SalaryClass'].value_counts()

1    137
0     97
2     84
Name: SalaryClass, dtype: int64

In [46]:
#join again using the formatted names and see how many matches now
formattedjoin = pd.merge(salaries,
                 scorecard,
                 on='Formatted School Name', 
                 how='inner')

formattedjoin.shape

(170, 136)

In [47]:
formattedjoin.head()

,School Name,Region,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary,School Name Extract 1,School Name Extract 2,...,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GT_25K_P6,GT_28K_P6,GRAD_DEBT_MDN_SUPP,GRAD_DEBT_MDN10YR_SUPP,RPY_3YR_RT_SUPP,C150_L4_POOLED_SUPP,C150_4_POOLED_SUPP
0,Stanford University,California,"$70,400.00",120,"$68,400.00","$93,100.00","$184,000.00","$257,000.00",Stanford University,Stanford University,...,0.1031,0.0119,94000,0.871,0.841,11446.5,121.687833161933,0.8992,NaN,0.9387
1,Harvey Mudd College,California,"$71,800.00",120,NaN,"$96,000.00","$180,000.00",NaN,Harvey Mudd College,Harvey Mudd College,...,0.3939,0.0000,88800,0.856,0.844,23500,249.828688184635,PrivacySuppressed,NaN,0.9252
2,"University of California, Berkeley",California,"$59,900.00",110,"$59,500.00","$81,000.00","$149,000.00","$201,000.00","University of California, Berkeley",University of California- Berkeley,...,0.2600,0.0613,64700,0.755,0.747,13200,140.32930570371,0.8016838167,NaN,0.9165
3,Occidental College,California,"$51,900.00",100,NaN,"$54,800.00","$157,000.00",NaN,Occidental College,Occidental College,...,0.4275,0.0040,50600,0.677,0.682,23215,246.798850902396,0.8759541985,NaN,0.8416
4,Pomona College,California,"$48,600.00",100,NaN,"$63,300.00","$161,000.00",NaN,Pomona College,Pomona College,...,0.1443,0.0042,58100,0.639,0.671,13400,142.455507305281,0.8510638298,NaN,0.9569


In [48]:
columnstokeep = ['SalaryClass','HCM2','PREDDEG','HIGHDEG','HBCU','PBI','MENONLY','WOMENONLY','SATVR25','SATVR75','SATMT25','SATMT75','SATWR25','SATWR75','SATVRMID','SATMTMID','SATWRMID','ACTCM25','ACTCM75','ACTEN25','ACTEN75','ACTMT25','ACTMT75','ACTWR25','ACTWR75','PCIP01','PCIP03','PCIP04','PCIP05','PCIP09','PCIP10','PCIP11','PCIP12','PCIP13','PCIP14','PCIP15','PCIP16','PCIP19','PCIP22','PCIP23','PCIP24','PCIP25','PCIP26','PCIP27','PCIP29','PCIP30','PCIP31','PCIP38','PCIP39','PCIP40','PCIP41','PCIP42','PCIP43','PCIP44','PCIP45','PCIP46','PCIP47','PCIP48','PCIP49','PCIP50','PCIP51','PCIP52','PCIP54','DISTANCEONLY','UGDS']

In [49]:
finaldf = formattedjoin[columnstokeep]

In [50]:
finaldf.shape

(170, 65)

In [51]:
finaldf['SalaryClass'].value_counts()

1    68
2    55
0    47
Name: SalaryClass, dtype: int64